# Análise do Dataset

1. Quais são os 15 produtos mais vendidos em termos de quantidade e receita?

2. Quais são os 15 produtos mais cancelados no período em termos de quantidade e receita?

3. Quais são os 15 produtos com mais ocorrências de cancelamento no período?

4. Qual são os paises com o maiores volumes de vendas? E o paises com o maiores valores médios de compra por transação?

5. Quais são os padrões sazonais nas vendas ao longo do ano?

6. Como as vendas variam ao longo da semana ou do mês?

7. Quais são os produtos mais frequentemente comprados em conjunto?

8. Existe uma correlação entre o tamanho do pedido (quantidade de itens) e o valor total da compra?

9. Quais são os clientes mais frequentes e quem são os maiores gastadores?

10. Qual é o valor médio de compra por cliente?

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configurando o estilo de plotagem do seaborn
sns.set_style('whitegrid')


In [3]:
# Carregando e lendo o dataset
sales = pd.read_csv('C:/Users/mathe/Projetos Dados/EcommerceSalesAnalysis/App/Data/Online Retail.csv', delimiter=';', encoding='latin1', decimal=',')
sales.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


In [ ]:
sales.shape

Uma análise do Dataset mostrou que além de registrar as operações de venda e cancelamento dos produtos, esse dataset também registra operações manuais de ajustes, descontos ofertados, taxas, comissões e outras operações.

Portanto, como a análise está focada apenas na VENDA e CANCELAMENTO dos PRODUTOS, os registros citados serão excluídos.

Essas operações são identificadas pela coluna *[StockCode]*, em que seus valores iniciam-se por uma letra.

In [ ]:
# Filtrando as operações que não se referem a produtos
def not_number(value):
    return not value[0].isdigit()

# Filtra todos os registros que não vendas/cancelamento de produtos
not_product_lines = sales[sales['StockCode'].apply(not_number)]

# Obter os valores únicos das linhas
not_product = not_product_lines['StockCode'].unique()

# Imprimir os valores únicos que não começam com números
print(not_product)

In [ ]:
# Filtrando o dataset apenas por venda e cancelamento de produtos
sales = sales[~sales['StockCode'].isin(not_product)]

In [ ]:
# Conhecendo os tipos dos dados
sales.dtypes

In [ ]:
# Alterando as colunas para os tipos de datas corretos
sales['InvoiceDate'] = pd.to_datetime(sales['InvoiceDate'], format = '%d/%m/%Y %H:%M').dt.date
sales.sample(5)

In [ ]:
# Adicionando coluna de Preço Final de venda/devolução/descontos ou taxas para cada transação
sales['FinalPrice'] = sales['Quantity'] * sales['UnitPrice']

In [ ]:
# Resumo dos dados
sales[['Quantity', 'UnitPrice', 'FinalPrice']].describe().T

In [ ]:
# Excluindo os registros com valor unitário igual a zero
sales = sales[sales['UnitPrice'] > 0]
sales.shape

In [ ]:
# Verifica a presença de registros duplicados no dataset
cols_list = sales.columns
duplicates = sales.duplicated(subset=cols_list, keep=False)
sales[duplicates].sort_values('Description').head()

In [ ]:
# Remove as linhas duplicadas e retorna o novo tamanho do dataset
sales.drop_duplicates(inplace=True)
sales.shape

In [ ]:
# Checando valores ausentes
sales.isna().sum()

Há presença de valores ausentes na coluna *[CustomerID].*

A coluna de *[CustomerID]* só será utilizada nas duas últimas análises, logo, as linhas com valores ausentes serão mantidas na análise inicial.

In [ ]:
# Verificando a existência de nomes de produtos iguais para diferentes StockCodes
duplicates_descriptions = sales.groupby('Description')['StockCode'].nunique()

# Filtra os casos onde há mais de um StockCode para a mesma Description
duplicates_descriptions = duplicates_descriptions[duplicates_descriptions > 1]

# Imprime as Description com mais de um StockCode associado
print("Descriptions com mais de um StockCode associado:")
print(duplicates_descriptions.sort_values(ascending=False))

In [ ]:
# Verificando a existência de StockCodes iguais para diferentes Description
duplicates_stock_codes = sales.groupby('StockCode')['Description'].nunique()

# Filtra os casos onde há mais de um Description para um mesmo StockCode
duplicates_stock_codes = duplicates_stock_codes[duplicates_stock_codes > 1]

# Imprime os StockCodes com mais de um Description associado
print("StockCodes com mais de um Description associado:")
print(duplicates_stock_codes.sort_values(ascending=False))

Há dois casos observados:
1. Um mesmo código possui descrições diferentes
2. Códigos diferentes possuem descrições iguais.

Para a análise feita neste notebook, será considerado que esses casos são decorrentes de cadastros errados, duplicados, registros incorretos ou alterações propositais nas descrições. 

In [ ]:
# Quantidade de valores únicos da coluna Description
sales.nunique()

Para simplificar a análise e melhorar a identificação dos produtos, cada registro único de *[Description]* será considerado como um produto diferente, já que a quantidade valores únicos de StockCode e Description são bem próximos.

Observa-se, portanto, um total de 4019 tipos de produtos distintos

### 1. Quais são os produtos mais vendidos em termos de quantidade e receita?
Como os valores negativos em *[Quantity]* e *[FinalPrice]* para as transações representam as devoluções, é importante considera-los para obter o saldo final da quantidade de produtos que realmente foi vendida e o valor real obtido por cada produto.

In [ ]:
# Agrupando os dados por produto, considerando a quantidade total vendida e receita gerada
sales_grouped = sales.groupby('Description').agg({'Quantity': 'sum', 'FinalPrice': 'sum'}).reset_index()
sales_grouped.head()

In [ ]:
def title_col(data, col):
    '''
    Converte a primeira letra de cada palavra para maíuscula em uma coluna específica de um dataframe
    
    Parâmetros:
    -----------    
    data : pandas.DataFrame
        O DataFrame onde a coluna está localizada
    col : str
        A coluna que será transformada. Deve ser string.
    '''
    data[col] = data[col].apply(lambda x: x.title())
    return data

In [ ]:
# Ordenando os produtos com mais unidades vendidas
sales_quantity_sorted = sales_grouped.sort_values(by='Quantity', ascending=False).reset_index(drop=True)

# Removendo maiúsculas desnecessárias
sales_quantity_sorted = title_col(sales_quantity_sorted, 'Description')

# Verificando se há apenas produtos nos primeiros 15 registros
sales_quantity_sorted[:15]

In [ ]:
# Ordenando os produtos que geraram mais receita
profitable_sales = sales_grouped.sort_values(by='FinalPrice', ascending=False).reset_index(drop=True)

# Removendo maiúsculas desnecessárias
profitable_sales = title_col(profitable_sales, 'Description')

# Verificando se há apenas produtos nos primeiros 15 registros
profitable_sales[:15]

In [ ]:
def format_number(num):
    '''
    Formata o número de acordo com o tipo de número recebido
    
    Parâmetro:
        num (int, float): Número a ser formatado
    '''
    if isinstance(num, float):
        if num.is_integer(): # Checa se a parte fracionária do float é zero
            return f"{int(num):d}"  # Transforma o float para int
        else:
            return f"{num:.2f}" # Retorna o float com 2 casas decimais
    if isinstance(num, int):
        return f"{num:d}"  # Retorna o número inteiro normalmente
    else:
        return str(num)  # Retorna como string se não for int nem float

In [ ]:
def add_bar_values(ax, space=5):
    '''
    Imprime os valores exatos de cada barra em um gráfico de barras horizontal
    
    Parâmetros:
        ax (matplotlib.axes.Axes): Gráfico a ser personalizado
        space (int, opcional): Espaçamento entre a barra e o início do texto. Padrão: 3
    '''
    for i in ax.patches: 
        bar_value = i.get_width() # Obtém o valor correspondente da barra, representado pela largura
        ax.text(bar_value + space, # Determina a posição horizontal de onde o valor será impresso
                i.get_y() + i.get_height() / 2, # Determina a posição vertical de onde o valor será impresso
                format_number(bar_value), # Valor que será impresso (formatado)
                ha='left', # Alinhamento horizontal do texto a esquerda
                va='center', # Alinhamento vertical do texto centralizado 
                fontsize=10) # Tamanho da fonte do número

In [ ]:
# Plotando o gráfico de produtos com mais unidades vendidas
plt.figure(figsize=(12,8))
top15_quantity = sns.barplot(data=sales_quantity_sorted.head(15), 
                             x='Quantity', y='Description', 
                             palette='viridis')
plt.xlabel('Quantidade Vendida', fontsize=14, fontweight='bold')
plt.ylabel('Produto', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Produtos Com Mais Unidades Vendidas no Período', fontsize=16, fontweight='bold')
add_bar_values(top15_quantity)
plt.show()

In [ ]:
# Plotando o gráfico de produtos que geraram mais receitas
plt.figure(figsize=(12,8))
top15_profitable = sns.barplot(data=profitable_sales.head(15), 
                               x='FinalPrice', y='Description', 
                               palette='viridis')

plt.xlabel('Valor Total (£)', fontsize=14, fontweight='bold')
plt.ylabel('Produto', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Produtos Mais Rentáveis no Período', fontsize=16, fontweight='bold')
add_bar_values(top15_profitable)
plt.show()

### 2. Quais são os produtos mais devolvidos no período em termos de quantidade e            receita?
As devoluções são pelos registros da coluna *[InvoiceNo]* que começam com C

In [ ]:
# Agrupando por produto, levando em conta ocorrências de cancelamento, quantidade cancelada e valor cancelado
canceled_products = sales[sales['InvoiceNo'].str.startswith('C')].groupby('Description')\
                        .agg({'Quantity':'sum', 'FinalPrice': 'sum', 'InvoiceNo':'count'})\
                        .abs().reset_index()

# Removendo maiúsculas desnecessárias
canceled_products = title_col(canceled_products, 'Description')

# Ordenando por produtos com mais unidades canceladas
more_units_canceled = canceled_products.sort_values(by=['Quantity'], ascending=False)\
                        .reset_index(drop=True)

# Ordenando por produtos com maiores valores de cancelamento
higher_canceled_values = canceled_products.sort_values(by=['FinalPrice'], ascending=False)\
                        .reset_index(drop=True)

# Ordenando por produtos com mais ocorrências de cancelamento
more_cancellations = canceled_products.sort_values(by=['InvoiceNo'], ascending=False)\
                        .reset_index(drop=True)

In [ ]:
# Checando os 20 primeiros produtos com mais unidades canceladas
more_units_canceled.head(20)

In [ ]:
# Checando os 20 primeiros produtos com maior valor de cancelamento
higher_canceled_values.head(20)

In [ ]:
# Checando os 20 primeiros produtos com maiores ocorrências de cancelamento
more_cancellations.head(20)

In [ ]:
# Plotando o gráfico de produtos com mais unidades canceladas
plt.figure(figsize=(12,8))
top15_canceled_units = sns.barplot(data=more_units_canceled.head(15), 
                               x='Quantity', y='Description', 
                               palette='rocket_r')

plt.xlabel('Unidades Canceladas', fontsize=14, fontweight='bold')
plt.ylabel('Produto', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Produtos Com Mais Unidades Canceladas no Período', fontsize=16, fontweight='bold')
add_bar_values(top15_canceled_units)
plt.show()

In [ ]:
# Plotando o gráfico de produtos com maiores valores de cancelamento
plt.figure(figsize=(12,8))
top15_canceled_values = sns.barplot(data=higher_canceled_values.head(15), 
                               x='FinalPrice', y='Description', 
                               palette='rocket_r')

plt.xlabel('Valor Total (£)', fontsize=14, fontweight='bold')
plt.ylabel('Produto', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Maiores Preços de Cancelamento no Período', fontsize=16, fontweight='bold')
add_bar_values(top15_canceled_values)
plt.show()

### 3. Quais são os 15 produtos com mais ocorrências de cancelamento no período?

In [ ]:
# Plotando o gráfico de produtos com mais ocorrências de cancelamento no período
plt.figure(figsize=(12,8))
top15_cancel_occurrences = sns.barplot(data=more_cancellations.head(15), 
                               x='InvoiceNo', y='Description', 
                               palette='rocket_r')

plt.xlabel('Ocorrências', fontsize=14, fontweight='bold')
plt.ylabel('Produto', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Produtos com Mais Ocorrências de Cancelamento', fontsize=16, fontweight='bold')
add_bar_values(top15_cancel_occurrences, 2)
plt.show()